[Introduction | 🦜️🔗 LangChain](https://python.langchain.com/v0.2/docs/introduction/)

In [ ]:
# !pip install -Uq langchain chromadb

In [ ]:
import os
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
# 첫 번째 텍스트 파일 찾기 함수
def get_first_txt_file(directory='.'):
    txt_files = [f for f in os.listdir(directory) if f.endswith('yp6WuHFhYCo_subtitle.txt')]
    txt_files.sort()  # 파일명 알파벳 순으로 정렬
    return txt_files[0] if txt_files else None

# 벡터 스토어 생성 함수
def create_vector_store():
    embeddings = OpenAIEmbeddings()
    return Chroma("langchain_store", embeddings)

# LangChain 기반 번역 함수
def translate_text(text):
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
    prompt = ChatPromptTemplate.from_template(
        "You are a translator. Translate the following text from English to Korean: {text}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(text=text)

# 번역 결과 파일로 저장
def save_translations_to_file(vector_store, output_file="langchain_translations.txt"):
    with open(output_file, "w", encoding="utf-8") as f:
        results = vector_store.similarity_search("", k=len(vector_store.get()))
        for doc in results:
            f.write(f"Original: {doc.page_content}\n")
            f.write(f"Translation: {doc.metadata['translation']}\n")
            f.write("---\n")
    print(f"번역 결과가 {output_file}에 저장되었습니다.")

# 번역 결과 검색 함수
def search_translations(vector_store, query, k=5):
    results = vector_store.similarity_search(query, k=k)
    print(f"'{query}'와 유사한 상위 {k}개 번역 결과:")
    for i, doc in enumerate(results, 1):
        print(f"{i}. Original: {doc.page_content}")
        print(f"   Translation: {doc.metadata['translation']}")
        print("---")

# 번역 통계 확인
def get_translation_statistics(vector_store):
    results = vector_store.similarity_search("", k=len(vector_store.get()))
    total_chars_original = sum(len(doc.page_content) for doc in results)
    total_chars_translated = sum(len(doc.metadata['translation']) for doc in results)
    avg_chars_original = total_chars_original / len(results)
    avg_chars_translated = total_chars_translated / len(results)
    
    print(f"총 번역된 청크 수: {len(results)}")
    print(f"원본 총 문자 수: {total_chars_original}")
    print(f"번역 총 문자 수: {total_chars_translated}")
    print(f"원본 평균 문자 수: {avg_chars_original:.2f}")
    print(f"번역 평균 문자 수: {avg_chars_translated:.2f}")

# 메인 함수
def main():
    first_txt_file = get_first_txt_file()
    if not first_txt_file:
        print("No .txt files found in the current directory.")
        return

    # 텍스트 파일 로드
    loader = TextLoader(first_txt_file, encoding='utf-8')
    documents = loader.load()
    
    # LangChain의 RecursiveCharacterTextSplitter 사용
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=0
    )
    chunks = text_splitter.split_documents(documents)
    
    print(f"파일 '{first_txt_file}'에서 총 {len(chunks)}개의 청크가 생성되었습니다.")

    # 벡터 스토어 생성
    vector_store = create_vector_store()

    print("번역을 시작합니다...")
    for i, chunk in enumerate(chunks):
        translated = translate_text(chunk.page_content)
        doc = Document(page_content=chunk.page_content, metadata={"translation": translated})
        vector_store.add_documents([doc])
        print(f"청크 {i+1}/{len(chunks)} 번역 완료")

    print("번역이 완료되었습니다.")

    # 결과 출력 (예시로 처음 5개 문서)
    results = vector_store.similarity_search("", k=5)
    for doc in results:
        print(f"Original: {doc.page_content}")
        print(f"Translation: {doc.metadata['translation']}")
        print("---")

if __name__ == "__main__":
    main()

In [ ]:
vector_store = create_vector_store()

# 번역 결과 파일로 저장
save_translations_to_file(vector_store)

# 특정 쿼리로 유사한 번역 검색
search_translations(vector_store, "PSF", k=3)

# 번역 통계 출력
get_translation_statistics(vector_store)

In [ ]:
results = vector_store.similarity_search("Python", k=len(vector_store.get()))

In [ ]:
results